# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7fe49f86ddd8>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()


['A']
['B']
['C']
-----
{'A'}
{'A'}
set()
-----
{'A'}
{'B'}
{'A'}
False
Allowed edges: 
('A', 'B')
Connected Nodes:  ['A', 'B']
Connected Edges:  [('A', 'B')]
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', 'C')
New node added--1
['A', 'B']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3]
-----
{'A'}
{'C'}
{'A'}
False
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Connected Nodes:  ['A', 'C']
Connected Edges:  [('A', 'C')]
Allowed edges: 
('B', 'C')
New node added--1
['A', 'C']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3, 4]
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', '

In [4]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

{(0, 3): 0, (0, 4): 1, (0, 6): 5, (1, 3): 0, (1, 5): 2, (1, 6): 3, (2, 4): 1, (2, 5): 2, (2, 6): 4, (3, 6): 4, (4, 6): 3, (5, 6): 5}


In [5]:
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = [1.0,1.0,1.0,1.0,1.0,1.0]*np.array([1e8])  #Poor yield Simruntime = 1sec

update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
vec_rn = VectorizedRxnNet(rn, dev='cpu')
print(vec_rn.kon)

{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 5}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 4}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=to

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [6]:
vec_rn.update_reaction_net(rn,1)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve()
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

[A + AB + ABC + AC - 1.0e-5, AB + ABC + B + BC - 1.0e-5, ABC + AC + BC + C - 1.0e-5, -100000000.0*A*B + 0.0274235939196359*AB, -100000000.0*A*C + 0.0274235939196359*AC, -100000000.0*B*C + 0.0274235939196359*BC, -100000000.0*AB*C + 7.52053503469088e-12*ABC, 7.52053503469088e-12*ABC - 100000000.0*AC*B, -100000000.0*A*BC + 7.52053503469088e-12*ABC]
Matrix([[6.138534e-9], [1.992939e-9], [6.684028e-9], [2.569674e-9], [-2.005246e-9], [2.030982e-9], [9.993331e-6]])
Equilibrium expected yield:  99.9333118670620 %
Parameter containing:
tensor([100000000., 100000000., 100000000., 100000000., 100000000., 100000000.],
       dtype=torch.float64, requires_grad=True)


In [7]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

A
B
C
AB
AC
BC
ABC
tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'B'}), 'kon': 100000000.00000018, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 0.027423593919635874, 'uid': 0}
tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'C'}), 'kon': 100000000.00000018, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 0.027423593919635874, 'uid': 1}
tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'C'}), 'kon': 100000000.00000018, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 0.027423593919635874, 'uid': 2}
tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'A', 'C'}), 'kon': 100000000.00000018, 'score': tensor([-100000.], dtype=torch.float64), 'koff': 7.520535034690883e-12, 'uid': 3}
tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'C'}, {'A', 'B'}), 'kon': 100000000.00000018

In [8]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*solution[edge[1]] - temp_kon*(solution[edge[0]])*(solution[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Free Energy: ",vec_rn.rxn_score_vec[uid])
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(solution[edge[0]])*(solution[node_map[react]])- koff*solution[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}

if solution != None:
    for n in rn.network.nodes():

        n_str = gtostr(rn.network.nodes[n]['struct']) 

        paths = [n_str]
        kon_sum = 0
        temp_node = n
        max_edge = True
        consumption_rate = 0
        if n < len(rn.network.nodes()):#num_monomers:
    #         print("Current node: ")
    #         print(n_str)
            while max_edge:
                max_edge = get_max_edge(temp_node)
                if max_edge:
                    total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                    temp_node = max_edge[1]
                    kon_sum += max_edge[0].item()


    #                 print("Next node: ")
    #                 print(temp_node)

                    paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
                else:
                    break
            pathway.append(paths)
            kon_sumarray.append(kon_sum)
            paths=[]

        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")
        print("|                                                                             |")
        node_flux = get_node_flux(n)
        net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
        print("|                                                                             |")
        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")

    print(pathway)
    print(kon_sumarray)
    #print(total_con_rate)

{'A': 0, 'B': 1, 'C': 2, 'AB': 3, 'AC': 4, 'BC': 5, 'ABC': 6}
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
|                                                                             |
Reaction:  A + B  ->  AB
Net flux:  -1.15290253511726e-9
kon :  tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
koff:  tensor([0.0274], dtype=torch.float64, grad_fn=<ExpBackward>)
Free Energy:  tensor(-50000., dtype=torch.float64)
Reaction data OUTWARD: 
{'k_on': 100000000.00000018, 'k_off': 0.027423593919635874, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 0}
Reaction:  A + C  ->  AC
Net flux:  -4.15800407378404e-9
kon :  tensor(100000000., dtype=torch.float64, grad_fn=<SelectBackward>)
koff:  tensor([0.0274], dtype=torch.float64, grad_fn=<ExpBackward>)
Free Energy:  tensor(-50000., dtype=torch.float64)
Reaction data OUTWARD: 
{'k_on': 100000000.

In [9]:
if solution != None:
    for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
        print(k," : ", v.item())

AB  :  100000000.0
AC  :  100000000.0
BC  :  100000000.0
A  :  300000000.0
B  :  300000000.0
C  :  300000000.0


In [10]:
if solution !=None:
    for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
        print(k," : ", v)



C  :  -7.15197047820678e-9
A  :  -6.55763189269247e-9
B  :  -2.02965831997614e-9
AB  :  -5.64674806633366e-10
BC  :  2.96637788809433e-11
ABC  :  2.56466934772856e-9
AC  :  4.55763735159728e-9


## Using the optimizer with a 1 second simulation runtime ##

In [11]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=10000,
                  learning_rate=new_kon[0]/1e3,
                  device='cpu')
optim.rn.update_reaction_net(rn)
optim.optimize()

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([100000000., 100000000., 100000000., 100000000., 100000000., 100000000.],
       dtype=torch.float64, requires_grad=True)]
Using CPU
Current time:  tensor(0.9997, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0007, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 0 was 52.4%
current params: tensor([1.0002e+08, 1.0002e+08, 1.0002e+08, 1.0000e+08, 1.0000e+08, 1.0001e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9996, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0006, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 1 was 52.4%
current params: tensor([1.0004e+08, 1.0004e+08, 1.0004e+08, 1.0001e+08, 1.0001e+08, 1.0001e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9995,

current params: tensor([1.0033e+08, 1.0043e+08, 1.0043e+08, 1.0009e+08, 1.0014e+08, 1.0009e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9995, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 22 was 52.4%
current params: tensor([1.0034e+08, 1.0045e+08, 1.0045e+08, 1.0010e+08, 1.0015e+08, 1.0010e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9994, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0004, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 23 was 52.4%
current params: tensor([1.0036e+08, 1.0047e+08, 1.0047e+08, 1.0010e+08, 1.0015e+08, 1.0010e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9993, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0003, dtype

current params: tensor([1.0064e+08, 1.0087e+08, 1.0087e+08, 1.0018e+08, 1.0028e+08, 1.0018e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9994, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 44 was 52.3%
current params: tensor([1.0066e+08, 1.0089e+08, 1.0089e+08, 1.0019e+08, 1.0029e+08, 1.0019e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9993, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 45 was 52.3%
current params: tensor([1.0067e+08, 1.0091e+08, 1.0091e+08, 1.0019e+08, 1.0030e+08, 1.0019e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9992, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0001, dtype

current params: tensor([1.0095e+08, 1.0132e+08, 1.0131e+08, 1.0027e+08, 1.0043e+08, 1.0027e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9992, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 66 was 52.3%
current params: tensor([1.0097e+08, 1.0134e+08, 1.0133e+08, 1.0028e+08, 1.0044e+08, 1.0028e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9991, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 67 was 52.3%
current params: tensor([1.0098e+08, 1.0136e+08, 1.0135e+08, 1.0028e+08, 1.0044e+08, 1.0028e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9999, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0009, dtype

current params: tensor([1.0126e+08, 1.0176e+08, 1.0176e+08, 1.0036e+08, 1.0058e+08, 1.0036e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0009, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 88 was 52.3%
current params: tensor([1.0127e+08, 1.0178e+08, 1.0178e+08, 1.0037e+08, 1.0058e+08, 1.0037e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9999, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0008, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 89 was 52.3%
current params: tensor([1.0129e+08, 1.0180e+08, 1.0180e+08, 1.0037e+08, 1.0059e+08, 1.0037e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9998, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0007, dtype

...

current params: tensor([1.7420e+08, 1.9945e+08, 1.9945e+08, 1.1490e+08, 1.2266e+08, 1.1490e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9996, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4686 was 48.8%
current params: tensor([1.7421e+08, 1.9947e+08, 1.9947e+08, 1.1490e+08, 1.2266e+08, 1.1490e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9996, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0003, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4687 was 48.8%
current params: tensor([1.7423e+08, 1.9950e+08, 1.9949e+08, 1.1490e+08, 1.2267e+08, 1.1490e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9995, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0003, d

current params: tensor([1.7457e+08, 1.9993e+08, 1.9993e+08, 1.1495e+08, 1.2273e+08, 1.1495e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9994, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0002, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4708 was 48.8%
current params: tensor([1.7459e+08, 1.9996e+08, 1.9995e+08, 1.1495e+08, 1.2274e+08, 1.1495e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9994, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4709 was 48.8%
current params: tensor([1.7461e+08, 1.9998e+08, 1.9997e+08, 1.1495e+08, 1.2274e+08, 1.1495e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9993, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0001, d

current params: tensor([1.7495e+08, 2.0042e+08, 2.0041e+08, 1.1500e+08, 1.2280e+08, 1.1500e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0007, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4730 was 48.8%
current params: tensor([1.7496e+08, 2.0044e+08, 2.0044e+08, 1.1500e+08, 1.2280e+08, 1.1500e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9999, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0007, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4731 was 48.8%
current params: tensor([1.7498e+08, 2.0046e+08, 2.0046e+08, 1.1501e+08, 1.2281e+08, 1.1501e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9999, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0006, d

current params: tensor([1.7532e+08, 2.0090e+08, 2.0090e+08, 1.1505e+08, 1.2287e+08, 1.1505e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9998, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4752 was 48.8%
current params: tensor([1.7534e+08, 2.0092e+08, 2.0092e+08, 1.1505e+08, 1.2287e+08, 1.1505e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9997, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0005, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 4753 was 48.8%
current params: tensor([1.7536e+08, 2.0094e+08, 2.0094e+08, 1.1506e+08, 1.2288e+08, 1.1506e+08],
       dtype=torch.float64)
Using CPU
Current time:  tensor(0.9997, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time:  tensor(1.0004, d

KeyboardInterrupt: 

In [ ]:
print(optim.yield_per_iter[-1])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)

In [ ]:
for n in rn.network.nodes():
    print(gtostr(rn.network.nodes[n]['struct']), " : ", n)

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','C','ABC','AB','BC','AC']
#nodes_list = ['AB','BMS','ABS','AMS','ABMS','AM','BM','MS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:

optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1, 4)
optim_rn = optim.rn
for i, runtime in enumerate([50, 100, 500,100000]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()

In [ ]:
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 